## Molecule Viewer

$$\require{mhchem}$$ 

#### Use drop downs to select molecule.  Display options are 'line' (not very visible), 'stick', and 'sphere', which is space-filling.
#### Can choose to view two molecules at same time, or just one with the check box.  In viewing box, scrolling mouse wheel zooms, and mouse-button motion rotates the molecule.

In [96]:
import py3Dmol, glob
import os.path
from rdkit import Chem
import ipywidgets as widgets

def MolTo3DView(mol, size=(300, 300), style="stick", surface=False, opacity=0.5):
    """Draw molecule in 3D
    
    Args:
    ----
        mol: rdMol, molecule to show
        size: tuple(int, int), canvas size
        style: str, type of drawing molecule
               style can be 'line', 'stick', 'sphere', 'carton'
        surface, bool, display SAS
        opacity, float, opacity of surface, range 0.0-1.0
    Return:
    ----
        viewer: py3Dmol.view, a class for constructing embedded 3Dmol.js views in ipython notebooks.
    """
    assert style in ('line', 'stick', 'sphere', 'cartoon')
    mblock = Chem.MolToMolBlock(mol)
    viewer = py3Dmol.view(width=size[0], height=size[1])
    viewer.addModel(mblock, 'mol')
    viewer.setStyle({style:{}})
    if surface:
        viewer.addSurface(py3Dmol.SAS, {'opacity': opacity})
    viewer.zoomTo()
    return viewer



In [98]:
out_layout = widgets.Layout(border = "1px solid black", width = "400px", height = "300px")
outbox = widgets.Output(layout = out_layout)
outbox2 = widgets.Output(layout = out_layout)

# dictionary to make molecules more viewable:
data = {
    'NH3': ("ammonia", "NH3"),
    'BrF5': ('brf5', 'BrF5'), 
    'CH4': ('ch4', "CH_4"),
    'ClF3': ('clf3', "ClF_3"),
    'H2CO': ('h2co', "H_2CO"),
    'IF5': ('if5', "IF_5"),
    'PCl5': ('pcl5', "PCl_5"),
    'SF4': ('sf4', "SF_4"),
    'SF6': ('sf6', "SF_6"),
    'SO2': ('sulfur_dioxide', "SO_2"),
    'XeF2': ('xef2', "XeF_2"),
    'XeF4': ('xef4', "XeF_4"), 
    'cis-[PtCl2(NH3)2]': ('cis_pt', "cis-[PtCl2(NH3)2]"),
    'trans-[PtCl2(NH3)2]': ('trans_pt', "trans-[PtCl2(NH3)2]"),
    'R-2-butanol': ('rbut_2ol', "R-CH_3CH_2CH(OH)CH_3"),
    'S-2-butanol': ('sbut_2ol', "S-CH_3CH_2CH(OH)CH_3"),
    'R-complex': ('tris_en1', "R-[Co(en)3]"),
    'S-complex': ('tris_en2', "S-[Co(en)3]"),
}

name_format = r"$$\ce{{ {} }}$$"
nice_name = widgets.HTMLMath(name_format.format("NH_3"))
nice_name2 = widgets.HTMLMath(name_format.format("NH_3"))

# two simple callback funtions:
def style_selector(change):
    style = drop_style.value
    outbox.clear_output(wait = True)
    name, display_text = data[drop_file.value]
    filename = os.path.join("data" , name + ".pdb")
    nice_name.value = name_format.format(display_text)
    m = Chem.MolFromPDBFile(filename, sanitize=False, removeHs=False)
    with outbox:
        display(MolTo3DView(m, style = style))
    
def style_selector2(change):
    style = drop_style2.value
    outbox2.clear_output(wait = True)
    name, display_text = data[drop_file2.value]
    filename = os.path.join("data" , name + ".pdb")
    nice_name2.value = name_format.format(display_text)
    m = Chem.MolFromPDBFile(filename, sanitize=False, removeHs=False)
    with outbox2:
        display(MolTo3DView(m, style = style))
        
drop_layout = widgets.Layout(height = "30px", width = "250px")
drop_file = widgets.Dropdown(options = data.keys(), description = "File",
                            layout = drop_layout)
drop_style = widgets.Dropdown(options = ['line', 'stick', 'sphere'], value = 'stick', description = 'Style:',
                             layout = drop_layout)
check = widgets.Checkbox(value = True, description ="show two molecules")

# output2
drop_file2 = widgets.Dropdown(options = data.keys(), description = "File", layout = drop_layout)
drop_style2 = widgets.Dropdown(options = ['line', 'stick', 'sphere'], value = 'stick', description = 'Style:',
                              layout = drop_layout)

placeholder = widgets.Label("")
row1 = widgets.HBox([drop_file, nice_name])
row2 = widgets.HBox([drop_style, placeholder])
box1 = widgets.VBox([row1, row2])

row21 = widgets.HBox([drop_file2, nice_name2])
row22 = widgets.HBox([drop_style2, placeholder])
box2 = widgets.VBox([row21, row22])

# main part boxes
box = widgets.VBox([box1, outbox ])
box2 = widgets.VBox([box2, outbox2 ])
main_row = widgets.HBox([box, box2])

#package = widgets.HBox([box, box2, check])
package = widgets.VBox([main_row, check])
display(package)

def showTwo(change):
    visibility = "visible" if check.value else "hidden"
    box2.layout.visibility = visibility
    
    
drop_file.observe(style_selector, names = 'value')
drop_style.observe(style_selector, names = 'value')
check.observe(showTwo, names = 'value')

drop_file2.observe(style_selector2, names = 'value')
drop_style2.observe(style_selector2, names = 'value')

style_selector("start")
style_selector2("start")

In [ ]:
%%html
<style>
div.input{
    display:none;
}
</style>